Notatki

Struktura
* Main - Query filmów
    * Wyniki w formie tabelki 
        * Podstrona danego filmu
            * Może wyświetlanie bar chartu wszystkich ocen
            *
* Podstrona analityczna 
    * Histogramy, porównanie gatunków, korelacja itd.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import sqlite3

In [89]:
# Import data
movie_data = pd.read_csv("movies_metadata.csv", low_memory=False)

In [91]:
# Removing 
i=0
size = movie_data.shape[0]
for i in range(0,size):
    restrictedList = ['True','False']
    if  movie_data.iloc[i,0] not in restrictedList:
        movie_data = movie_data.drop(movie_data.index[[i-1]])
        movie_data = movie_data.drop(movie_data.index[[i-1]])

In [92]:
# Drop columns
movie_data = movie_data.drop(['popularity','production_countries', 'belongs_to_collection', 'homepage', 'spoken_languages', 'original_language', 'video', 'production_companies', 'original_title'], axis=1)

In [93]:
# Extract first name in genres
plus_size = movie_data.shape[0]+1
i = 0
for i in range(0,plus_size):
    try:
        movie_data.iloc[plus_size - i,2] = movie_data.iloc[plus_size - i,2].split()[3][1:-3]
    except IndexError:
        next

for i in range(0,size):
    restrictedList = ['[]','','i','','88']
    if movie_data.iloc[i,2] in restrictedList:
        movie_data.iloc[i,2] = 'No genre'

In [101]:
movie_data.drop_duplicates(subset=['id'],inplace=True)
movie_data = movie_data.drop([0])

In [103]:
movie_data

,adult,budget,genres,id,imdb_id,overview,poster_path,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count
1,False,65000000,Adventure,8844,tt0113497,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,False,0,Romance,15602,tt0113228,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,False,16000000,Comedy,31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,False,0,Comedy,11862,tt0113041,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
5,False,60000000,Action,949,tt0113277,"Obsessive master thief, Neil McCauley leads a ...",/zMyfPUelumio3tiDKPffaUpsQTD.jpg,1995-12-15,187436818.0,170.0,Released,A Los Angeles Crime Saga,Heat,7.7,1886.0
6,False,58000000,Comedy,11860,tt0114319,An ugly duckling having undergone a remarkable...,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,1995-12-15,0.0,127.0,Released,You are cordially invited to the most surprisi...,Sabrina,6.2,141.0
7,False,0,Action,45325,tt0112302,"A mischievous young boy, Tom Sawyer, witnesses...",/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,1995-12-22,0.0,97.0,Released,The Original Bad Boys.,Tom and Huck,5.4,45.0
8,False,35000000,Action,9091,tt0114576,International action superstar Jean Claude Van...,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,1995-12-22,64350171.0,106.0,Released,Terror goes into overtime.,Sudden Death,5.5,174.0
9,False,58000000,Adventure,710,tt0113189,James Bond must unmask the mysterious head of ...,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,1995-11-16,352194034.0,130.0,Released,No limits. No fears. No substitutes.,GoldenEye,6.6,1194.0
10,False,62000000,Comedy,9087,tt0112346,"Widowed U.S. president Andrew Shepherd, one of...",/lymPNGLZgPHuqM29rKMGV46ANij.jpg,1995-11-17,107879496.0,106.0,Released,Why can't the most powerful man in the world h...,The American President,6.5,199.0


In [104]:
# Changing types of columns
movie_data[['budget', 'id','revenue','vote_average','vote_count']].astype(float)

,budget,id,revenue,vote_average,vote_count
1,65000000.0,8844.0,262797249.0,6.9,2413.0
2,0.0,15602.0,0.0,6.5,92.0
3,16000000.0,31357.0,81452156.0,6.1,34.0
4,0.0,11862.0,76578911.0,5.7,173.0
5,60000000.0,949.0,187436818.0,7.7,1886.0
6,58000000.0,11860.0,0.0,6.2,141.0
7,0.0,45325.0,0.0,5.4,45.0
8,35000000.0,9091.0,64350171.0,5.5,174.0
9,58000000.0,710.0,352194034.0,6.6,1194.0
10,62000000.0,9087.0,107879496.0,6.5,199.0


In [105]:
movie_data.to_csv('movie_data_cleaned.csv', sep=';', encoding='utf-8')

In [106]:
movie_data_c = pd.read_csv("movie_data_cleaned.csv", low_memory=False, sep=";")

In [107]:
movie_data_c.head(5)

,Unnamed: 0,adult,budget,genres,id,imdb_id,overview,poster_path,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count
0,1,False,65000000,Adventure,8844,tt0113497,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
1,2,False,0,Romance,15602,tt0113228,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
2,3,False,16000000,Comedy,31357,tt0114885,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
3,4,False,0,Comedy,11862,tt0113041,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
4,5,False,60000000,Action,949,tt0113277,"Obsessive master thief, Neil McCauley leads a ...",/zMyfPUelumio3tiDKPffaUpsQTD.jpg,1995-12-15,187436818.0,170.0,Released,A Los Angeles Crime Saga,Heat,7.7,1886.0


In [108]:
conn = sqlite3.connect('main_database.db')

c = conn.cursor()

query = """ 
CREATE TABLE movie_database_c (
    adult        BOOLEAN,
    budget       INTEGER,
    genres       VARCHAR (255),
    id           INTEGER       PRIMARY KEY
                               UNIQUE,
    imdb_id      INTEGER       UNIQUE,
    overview     TEXT,
    poster_path  VARCHAR (255),
    release_date DATE,
    revenue      DOUBLE,
    runtime      DOUBLE,
    status       VARCHAR (255),
    tagline      TEXT,
    title        VARCHAR (255),
    vote_average DOUBLE,
    vote_count   INTEGER
);"""

# Create table
c.execute(query)

# Save (commit) the changes in database.
# Changes not commited will be lost
conn.commit()

# We close the connection and free all resources
conn.close()



In [109]:
conn = sqlite3.connect('main_database.db')
c = conn.cursor()

# Create table
c.executemany('INSERT INTO movie_database_c VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', movie_data.values.tolist())

# Save (commit) the changes in database.
# Changes not commited will be lost
conn.commit()

# We close the connection and free all resources
conn.close()

In [ ]:
movie_data['adult'].value_counts()